In [5]:
import torch.nn as nn
from torchvision.models import resnet18, ResNet50_Weights
import os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import librosa
import torch


from torchvision import models

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

torch.cuda.empty_cache()


In [6]:

resnet50 = resnet18(pretrained=True)




class AudioClassificationDataset(Dataset):
    audioToSpecDic = dict(
      n_mels=224,
      hop_length=300,
      win_length=500,
      n_fft=1024,
      fmax=22000,
    )

    def __init__(self, x, y):
      self.x = x
      self.y = y


      # nn.Conv2d(1,3,(1,1)),
      # resnet18(),
      # nn.ReLU(),
      # nn.Linear(1000, 10)
            

    def audio_to_spectrogram(self, file):      # convertir audio a melspectrogram

      
      audio, fs = librosa.load(f'./audios-dataset/audio/{file}')
      spectogram = librosa.feature.melspectrogram(y=audio,sr=fs ,**self.audioToSpecDic)
      spectogram = librosa.power_to_db(spectogram)
      
      return librosa.util.normalize(spectogram)

    def __len__(self):
      return len(self.x)
  

    def __getitem__(self, idx):
      # acceder al audio
      # pasarlo melspectram+
      # falta un from numpy torch.reshape(224,224)
      x_audio = self.x[idx]
      x_img = self.audio_to_spectrogram(x_audio)
      x_img= torch.reshape(torch.from_numpy(x_img), (1, x_img.shape[0],x_img.shape[1]))
      x_img = x_img.repeat(3, 1, 1)
    
      return x_img, self.y[idx]  
      
      
      


/home/adria/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/adria/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            print('holaa')
            print(X.shape)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss

In [8]:
from sklearn.model_selection import train_test_split

In [10]:
def create_model():
  model = models.resnet50(pretrained=True)
  # model.fc = nn.Conv2d(1,3,(1,1))
  # model.fc = nn.ReLU()
  model.fc = nn.Linear(2048, 50)
  
  return model

def main():
  dataset = pd.read_csv('./audios-dataset/meta/esc50.csv', usecols=['filename', 'target'])
  train, test = train_test_split(dataset, test_size=0.2)
  print('device', device)
  # train = train[:1]

  batch_size = 5
  epochs = 20
  learning_rate = 0.003

  loss_fn = nn.CrossEntropyLoss()

  model = create_model().to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


  train_dataset = AudioClassificationDataset(train['filename'].values, train['target'].values)
  test_dataset = AudioClassificationDataset(test['filename'].values, test['target'].values)

  train_dataloader = DataLoader(train_dataset, batch_size , shuffle=True)
  test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
  

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")

    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test_loop(test_dataloader, model, loss_fn)

main()



  
  

device cuda


/home/adria/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/adria/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1
-------------------------------


/home/adria/.local/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


loss: 4.062785  [    0/ 1600]


KeyboardInterrupt: 